In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [2]:
load_dotenv()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [ ]:
# 1. Pull live data of chosen crypto every 24h
# 2. Manipulate data to run through model
# 3. Intiate buy/sell based on signal
# 4. Loop

In [22]:
# User inputs
crypto = 'BTCUSD'
investment = 1000

# Derived inputs
investment_increment = investment/20
amount = api.get_latest_crypto_bar(crypto, 'CBSE')
# amount = amount of chosen crypto to invest that equals the investment increment


BarV2({   'c': 21688.16,
    'h': 21690.85,
    'l': 21684.66,
    'n': 310,
    'o': 21688.8,
    't': '2022-08-25T11:47:00Z',
    'v': 3.47096586,
    'vw': 21688.011096149,
    'x': 'CBSE'})

In [ ]:
buy_order = api.submit_order(
    symbol=crypto,
    qty=amount
)